# IR Project

In [1]:
import numpy as np
import pandas as pd 
from collections import defaultdict
from pprint import pprint
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
 
matplotlib.rcParams['figure.figsize'] = (8, 6)

import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Merge
from keras.layers import Convolution1D
from keras.layers import Bidirectional
from keras.layers import Reshape
from keras.layers import Flatten
seed = 0
np.random.seed(seed)

import time
import operator
from datetime import datetime

In [2]:
cat_col = ['fecha_dato', 'ncodpers','ind_empleado','pais_residencia','sexo','age','fecha_alta','ind_nuevo','antiguedad','indrel', 'indrel_1mes','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall', 'tipodom','cod_prov','ind_actividad_cliente','renta','segmento']

notuse = ["ult_fec_cli_1t","nomprov"]

product_col = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

product_col = product_col[2:]

train_cols = cat_col + product_col


In [3]:
df_train = pd.read_csv('input/train_ver2.csv',usecols=train_cols)

/Users/rkcosmos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
df_train['fecha_dato'] = pd.to_datetime(df_train['fecha_dato'], format='%Y-%m-%d', errors='ignore')

In [6]:
month = 6

df_train_curr = df_train.loc[df_train['fecha_dato']=='2015-06-28',:]
df_train_5 = df_train.loc[df_train['fecha_dato']=='2015-05-28', product_col+['ncodpers']]
df_train_4 = df_train.loc[df_train['fecha_dato']=='2015-04-28', product_col+['ncodpers']]
df_train_3 = df_train.loc[df_train['fecha_dato']=='2015-03-28', product_col+['ncodpers']]
df_train_2 = df_train.loc[df_train['fecha_dato']=='2015-02-28', product_col+['ncodpers']]
df_train_1 = df_train.loc[df_train['fecha_dato']=='2015-01-28', product_col+['ncodpers']]

In [7]:
dfm = pd.merge(df_train_curr,df_train_5, how='left', on=['ncodpers'], suffixes=('', '_5'))
dfm = pd.merge(dfm,df_train_4, how='left', on=['ncodpers'], suffixes=('', '_4'))
dfm = pd.merge(dfm,df_train_3, how='left', on=['ncodpers'], suffixes=('', '_3'))
dfm = pd.merge(dfm,df_train_2, how='left', on=['ncodpers'], suffixes=('', '_2'))
dfm = pd.merge(dfm,df_train_1, how='left', on=['ncodpers'], suffixes=('', '_1'))

In [8]:
dfm.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,ind_cco_fin_ult1_5,ind_cder_fin_ult1_5,ind_cno_fin_ult1_5,ind_ctju_fin_ult1_5,ind_ctma_fin_ult1_5,ind_ctop_fin_ult1_5,ind_ctpp_fin_ult1_5,ind_deco_fin_ult1_5,ind_deme_fin_ult1_5,ind_dela_fin_ult1_5,ind_ecue_fin_ult1_5,ind_fond_fin_ult1_5,ind_hip_fin_ult1_5,ind_plan_fin_ult1_5,ind_pres_fin_ult1_5,ind_reca_fin_ult1_5,ind_tjcr_fin_ult1_5,ind_valo_fin_ult1_5,ind_viv_fin_ult1_5,ind_nomina_ult1_5,ind_nom_pens_ult1_5,ind_recibo_ult1_5,ind_cco_fin_ult1_4,ind_cder_fin_ult1_4,ind_cno_fin_ult1_4,ind_ctju_fin_ult1_4,ind_ctma_fin_ult1_4,ind_ctop_fin_ult1_4,ind_ctpp_fin_ult1_4,ind_deco_fin_ult1_4,ind_deme_fin_ult1_4,ind_dela_fin_ult1_4,ind_ecue_fin_ult1_4,ind_fond_fin_ult1_4,ind_hip_fin_ult1_4,ind_plan_fin_ult1_4,ind_pres_fin_ult1_4,ind_reca_fin_ult1_4,ind_tjcr_fin_ult1_4,ind_valo_fin_ult1_4,ind_viv_fin_ult1_4,ind_nomina_ult1_4,ind_nom_pens_ult1_4,ind_recibo_ult1_4,ind_cco_fin_ult1_3,ind_cder_fin_ult1_3,ind_cno_fin_ult1_3,ind_ctju_fin_ult1_3,ind_ctma_fin_ult1_3,ind_ctop_fin_ult1_3,ind_ctpp_fin_ult1_3,ind_deco_fin_ult1_3,ind_deme_fin_ult1_3,ind_dela_fin_ult1_3,ind_ecue_fin_ult1_3,ind_fond_fin_ult1_3,ind_hip_fin_ult1_3,ind_plan_fin_ult1_3,ind_pres_fin_ult1_3,ind_reca_fin_ult1_3,ind_tjcr_fin_ult1_3,ind_valo_fin_ult1_3,ind_viv_fin_ult1_3,ind_nomina_ult1_3,ind_nom_pens_ult1_3,ind_recibo_ult1_3,ind_cco_fin_ult1_2,ind_cder_fin_ult1_2,ind_cno_fin_ult1_2,ind_ctju_fin_ult1_2,ind_ctma_fin_ult1_2,ind_ctop_fin_ult1_2,ind_ctpp_fin_ult1_2,ind_deco_fin_ult1_2,ind_deme_fin_ult1_2,ind_dela_fin_ult1_2,ind_ecue_fin_ult1_2,ind_fond_fin_ult1_2,ind_hip_fin_ult1_2,ind_plan_fin_ult1_2,ind_pres_fin_ult1_2,ind_reca_fin_ult1_2,ind_tjcr_fin_ult1_2,ind_valo_fin_ult1_2,ind_viv_fin_ult1_2,ind_nomina_ult1_2,ind_nom_pens_ult1_2,ind_recibo_ult1_2,ind_cco_fin_ult1_1,ind_cder_fin_ult1_1,ind_cno_fin_ult1_1,ind_ctju_fin_ult1_1,ind_ctma_fin_ult1_1,ind_ctop_fin_ult1_1,ind_ctpp_fin_ult1_1,ind_deco_fin_ult1_1,ind_deme_fin_ult1_1,ind_dela_fin_ult1_1,ind_ecue_fin_ult1_1,ind_fond_fin_ult1_1,ind_hip_fin_ult1_1,ind_plan_fin_ult1_1,ind_pres_fin_ult1_1,ind_reca_fin_ult1_1,ind_tjcr_fin_ult1_1,ind_valo_fin_ult1_1,ind_viv_fin_ult1_1,ind_nomina_ult1_1,ind_nom_pens_ult1_1,ind_recibo_ult1_1
0,2015-06-28,16132,N,ES,V,48,1995-03-08,0.0,244,1.0,1,A,S,N,NaN,KAT,N,1.0,28.0,0.0,160900.95,02 - PARTICULARES,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-06-28,1063040,N,ES,V,25,2012-09-19,0.0,34,1.0,1,A,S,N,NaN,KHE,N,1.0,46.0,1.0,74693.67,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2015-06-28,1063041,N,ES,H,24,

In [9]:
prevcols = [col for col in dfm.columns if '_ult1_'+str(month-1) in col]
currcols = [col for col in dfm.columns if '_ult1' == col[-5:]]

In [10]:
all_product_col = [col for col in dfm.columns if '_ult1' in col]

for col in all_product_col:
    dfm[col].fillna(0, inplace=True)

In [11]:
for col in currcols:
    dfm[col] = dfm[col] - dfm[col+'_'+str(month-1)]
    dfm[col] = dfm[col].apply(lambda x: max(x,0))

In [12]:
dfm = dfm[dfm[currcols].sum(axis=1) >0]

In [13]:
print(dfm[currcols].sum(axis = 1).value_counts())

1.0    29492
2.0     5641
3.0     1412
4.0      156
5.0        9
dtype: int64


In [14]:
dfm = dfm.reset_index(drop=True) 
print(dfm.shape)

(36710, 154)


In [15]:
data = []

for index, row in dfm.iterrows():
    if index%1000 == 0:
        print(index)
        print('finish: ', time.strftime('%a %H:%M:%S'))
    if row[currcols].sum() > 0:
        for i,col in enumerate(currcols):
            if row[col] == 1:
                row['target'] = float(currcols.index(col))
                data.append(list(row))
                
df_new = pd.DataFrame(data, columns = list(dfm.columns.values) + ['target'])

0
finish:  Fri 10:31:19
1000
finish:  Fri 10:31:21
2000
finish:  Fri 10:31:23
3000
finish:  Fri 10:31:24
4000
finish:  Fri 10:31:26
5000
finish:  Fri 10:31:28
6000
finish:  Fri 10:31:29
7000
finish:  Fri 10:31:31
8000
finish:  Fri 10:31:32
9000
finish:  Fri 10:31:34
10000
finish:  Fri 10:31:36
11000
finish:  Fri 10:31:38
12000
finish:  Fri 10:31:39
13000
finish:  Fri 10:31:41
14000
finish:  Fri 10:31:43
15000
finish:  Fri 10:31:44
16000
finish:  Fri 10:31:46
17000
finish:  Fri 10:31:48
18000
finish:  Fri 10:31:49
19000
finish:  Fri 10:31:51
20000
finish:  Fri 10:31:53
21000
finish:  Fri 10:31:55
22000
finish:  Fri 10:31:56
23000
finish:  Fri 10:31:58
24000
finish:  Fri 10:31:59
25000
finish:  Fri 10:32:01
26000
finish:  Fri 10:32:02
27000
finish:  Fri 10:32:04
28000
finish:  Fri 10:32:06
29000
finish:  Fri 10:32:07
30000
finish:  Fri 10:32:09
31000
finish:  Fri 10:32:10
32000
finish:  Fri 10:32:12
33000
finish:  Fri 10:32:13
34000
finish:  Fri 10:32:15
35000
finish:  Fri 10:32:16
36000

In [16]:
df_new.drop(currcols+['fecha_dato'], axis=1, inplace=True)

In [17]:
df_new.shape

(45679, 132)

In [ ]:
print(df_new['target'].value_counts())

0.0     9457
21.0    9131
20.0    8229
19.0    5161
16.0    4755
15.0    2942
2.0     1934
10.0    1219
9.0     1085
7.0      503
4.0      349
11.0     246
5.0      222
17.0     159
6.0      154
3.0       55
8.0       33
13.0      21
1.0        9
14.0       8
12.0       4
18.0       3
Name: target, dtype: int64


In [2]:
cat_col = ['fecha_dato', 'ncodpers','ind_empleado','pais_residencia','sexo','age','fecha_alta','ind_nuevo','antiguedad','indrel', 'indrel_1mes','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall', 'tipodom','cod_prov','ind_actividad_cliente','renta','segmento']

notuse = ["ult_fec_cli_1t","nomprov"]

product_col = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

product_col = product_col[2:]

train_cols = cat_col + product_col


In [3]:
df_train = pd.read_csv('input/train_ver2.csv',usecols=train_cols)

/Users/rkcosmos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
pd.set_option('display.max_columns', None)

In [6]:
month = 6

df_train_5 = df_train.loc[df_train['fecha_dato']=='2016-05-28', product_col+['ncodpers']]
df_train_4 = df_train.loc[df_train['fecha_dato']=='2016-04-28', product_col+['ncodpers']]
df_train_3 = df_train.loc[df_train['fecha_dato']=='2016-03-28', product_col+['ncodpers']]
df_train_2 = df_train.loc[df_train['fecha_dato']=='2016-02-28', product_col+['ncodpers']]
df_train_1 = df_train.loc[df_train['fecha_dato']=='2016-01-28', product_col+['ncodpers']]

In [7]:
dfm = pd.merge(dfm,df_train_4, how='left', on=['ncodpers'], suffixes=('', '_4'))
dfm = pd.merge(dfm,df_train_3, how='left', on=['ncodpers'], suffixes=('', '_3'))
dfm = pd.merge(dfm,df_train_2, how='left', on=['ncodpers'], suffixes=('', '_2'))
dfm = pd.merge(dfm,df_train_1, how='left', on=['ncodpers'], suffixes=('', '_1'))

#prevcols = [col for col in dfm.columns if '_ult1_'+str(month-1) in col]
#currcols = [col for col in dfm.columns if '_ult1' == col[-5:]]

all_product_col = [col for col in dfm.columns if '_ult1' in col]

for col in all_product_col:
    dfm[col].fillna(0, inplace=True)

In [10]:
dfm.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_cco_fin_ult1_4,ind_cder_fin_ult1_4,ind_cno_fin_ult1_4,ind_ctju_fin_ult1_4,ind_ctma_fin_ult1_4,ind_ctop_fin_ult1_4,ind_ctpp_fin_ult1_4,ind_deco_fin_ult1_4,ind_deme_fin_ult1_4,ind_dela_fin_ult1_4,ind_ecue_fin_ult1_4,ind_fond_fin_ult1_4,ind_hip_fin_ult1_4,ind_plan_fin_ult1_4,ind_pres_fin_ult1_4,ind_reca_fin_ult1_4,ind_tjcr_fin_ult1_4,ind_valo_fin_ult1_4,ind_viv_fin_ult1_4,ind_nomina_ult1_4,ind_nom_pens_ult1_4,ind_recibo_ult1_4,ind_cco_fin_ult1_3,ind_cder_fin_ult1_3,ind_cno_fin_ult1_3,ind_ctju_fin_ult1_3,ind_ctma_fin_ult1_3,ind_ctop_fin_ult1_3,ind_ctpp_fin_ult1_3,ind_deco_fin_ult1_3,ind_deme_fin_ult1_3,ind_dela_fin_ult1_3,ind_ecue_fin_ult1_3,ind_fond_fin_ult1_3,ind_hip_fin_ult1_3,ind_plan_fin_ult1_3,ind_pres_fin_ult1_3,ind_reca_fin_ult1_3,ind_tjcr_fin_ult1_3,ind_valo_fin_ult1_3,ind_viv_fin_ult1_3,ind_nomina_ult1_3,ind_nom_pens_ult1_3,ind_recibo_ult1_3,ind_cco_fin_ult1_2,ind_cder_fin_ult1_2,ind_cno_fin_ult1_2,ind_ctju_fin_ult1_2,ind_ctma_fin_ult1_2,ind_ctop_fin_ult1_2,ind_ctpp_fin_ult1_2,ind_deco_fin_ult1_2,ind_deme_fin_ult1_2,ind_dela_fin_ult1_2,ind_ecue_fin_ult1_2,ind_fond_fin_ult1_2,ind_hip_fin_ult1_2,ind_plan_fin_ult1_2,ind_pres_fin_ult1_2,ind_reca_fin_ult1_2,ind_tjcr_fin_ult1_2,ind_valo_fin_ult1_2,ind_viv_fin_ult1_2,ind_nomina_ult1_2,ind_nom_pens_ult1_2,ind_recibo_ult1_2,ind_cco_fin_ult1_1,ind_cder_fin_ult1_1,ind_cno_fin_ult1_1,ind_ctju_fin_ult1_1,ind_ctma_fin_ult1_1,ind_ctop_fin_ult1_1,ind_ctpp_fin_ult1_1,ind_deco_fin_ult1_1,ind_deme_fin_ult1_1,ind_dela_fin_ult1_1,ind_ecue_fin_ult1_1,ind_fond_fin_ult1_1,ind_hip_fin_ult1_1,ind_plan_fin_ult1_1,ind_pres_fin_ult1_1,ind_reca_fin_ult1_1,ind_tjcr_fin_ult1_1,ind_valo_fin_ult1_1,ind_viv_fin_ult1_1,ind_nomina_ult1_1,ind_nom_pens_ult1_1,ind_recibo_ult1_1,ind_cco_fin_ult1_5,ind_cder_fin_ult1_5,ind_cno_fin_ult1_5,ind_ctju_fin_ult1_5,ind_ctma_fin_ult1_5,ind_ctop_fin_ult1_5,ind_ctpp_fin_ult1_5,ind_deco_fin_ult1_5,ind_deme_fin_ult1_5,ind_dela_fin_ult1_5,ind_ecue_fin_ult1_5,ind_fond_fin_ult1_5,ind_hip_fin_ult1_5,ind_plan_fin_ult1_5,ind_pres_fin_ult1_5,ind_reca_fin_ult1_5,ind_tjcr_fin_ult1_5,ind_valo_fin_ult1_5,ind_viv_fin_ult1_5,ind_nomina_ult1_5,ind_nom_pens_ult1_5,ind_recibo_ult1_5
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,1.0,A,S,N,N,KAT,N,1,28.0,1,326124.90,01 - TOP,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,1.0,I,S,N,NaN,KAT,N,1,3.0,0,NA,02 - PARTICULARES,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,1.0,A,S,N,NaN,KHE,N,1,15.0,1,NA,03 - UNIVERSITARIO,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,1.0,I,S,N,NaN,KHE,N,1,8.0,0,148402.98,03 - UNIVERSITARIO,1.0,0.0,0.0,0.0,0.0,

In [ ]:
dfm.drop('fecha_dato', axis=1, inplace=True)

In [ ]:
df_train = df_new
df_test = dfm

In [8]:
demographic_cols = ['ncodpers','fecha_alta','ind_empleado','pais_residencia','sexo','age','ind_nuevo','antiguedad','indrel',
 'indrel_1mes','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall',
 'tipodom','cod_prov','ind_actividad_cliente','renta','segmento']

notuse = ["ult_fec_cli_1t","nomprov",'fecha_dato']

product_col = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
drop_column = ['ind_nuevo','indrel','indresi','indfall','tipodom','ind_empleado','pais_residencia','indrel_1mes','indext','conyuemp','fecha_alta','tiprel_1mes']

df_train.drop(drop_column, axis=1, inplace = True)
df_test.drop(drop_column, axis=1, inplace = True)

In [7]:
df_test["renta"]   = pd.to_numeric(df_test["renta"], errors="coerce")
unique_prov = df_test[df_test.cod_prov.notnull()].cod_prov.unique()
grouped = df_test.groupby("cod_prov")["renta"].median()

def impute_renta(df):
    df["renta"]   = pd.to_numeric(df["renta"], errors="coerce")       
    for cod in unique_prov:
        df.loc[df['cod_prov']==cod,['renta']] = df.loc[df['cod_prov']==cod,['renta']].fillna({'renta':grouped[cod]}).values
    df.renta.fillna(df_test["renta"].median(), inplace=True)
    
impute_renta(df_train)
impute_renta(df_test)

In [8]:
def drop_na(df):
    df.dropna(axis = 0, subset = ['ind_actividad_cliente'], inplace = True) 
    
drop_na(df_train)

In [9]:
dummy_col = ['sexo','canal_entrada','cod_prov','segmento']
dummy_col_select = ['canal_entrada','cod_prov']

In [10]:
limit = int(0.05 * len(df_train.index))
use_dummy_col = {}

for col in dummy_col_select:
    trainlist = df_train[col].value_counts()
    use_dummy_col[col] = []
    for i,item in enumerate(trainlist):
        if item > limit:
            use_dummy_col[col].append(df_train[col].value_counts().index[i])   

In [11]:
def get_dummy(df):
    for col in dummy_col_select:
        for item in df[col].unique(): 
            if item not in use_dummy_col[col]:
                row_index = df[col] == item
                df.loc[row_index,col] = np.nan
    return pd.get_dummies(df, prefix=dummy_col, columns = dummy_col)
    
df_train = get_dummy(df_train)
df_test = get_dummy(df_test)

In [12]:
def clean_quantitative_param(df):
    df["age"]   = pd.to_numeric(df["age"], errors="coerce")
    max_age = 80 
    log_max_age = np.log(max_age) 
    square_max_age  = np.square(max_age)
    df["age"]   = df['age'].apply(lambda x: min(x ,max_age))
    df["log_age"]   = df['age'].apply(lambda x: round(np.log10(x+1)/log_max_age, 6))
    df["square_age"]   = df['age'].apply(lambda x: round(np.square(x)/square_max_age, 6))
    df["age"]   = df['age'].apply(lambda x: round( x/max_age, 6))
    
    max_renta = 1.0e6
    log_max_renta = np.log(max_renta) 
    square_max_renta  = np.square(max_renta)
    df["renta"]   = df['renta'].apply(lambda x: min(x ,max_renta))
    df["log_renta"]   = df['renta'].apply(lambda x: round(np.log10(x+1)/log_max_renta, 6))
    df["square_renta"]   = df['renta'].apply(lambda x: round(np.square(x)/square_max_renta, 6))
    df["renta"]   = df['renta'].apply(lambda x: round( x/max_renta, 6))
    
    df["antiguedad"]   = pd.to_numeric(df["antiguedad"], errors="coerce")
    df["antiguedad"] = df["antiguedad"].replace(-999999, df['antiguedad'].median())
    max_antigue = 256
    log_max_antigue = np.log(max_antigue) 
    square_max_antigue  = np.square(max_antigue)
    df["antiguedad"]   = df['antiguedad'].apply(lambda x: min(x ,max_antigue))
    df["log_antiguedad"]   = df['antiguedad'].apply(lambda x: round(np.log10(x+1)/log_max_antigue, 6))
    df["square_antiguedad"]   = df['antiguedad'].apply(lambda x: round(np.square(x)/square_max_antigue, 6))
    df["antiguedad"]   = df['antiguedad'].apply(lambda x: round( x/max_antigue, 6)) 
    
    return df      

In [13]:
df_train = clean_quantitative_param(df_train)
df_test = clean_quantitative_param(df_test)

In [14]:
product_col_5 = [col for col in df_train.columns if '_ult1_5' in col]
product_col_4 = [col for col in df_train.columns if '_ult1_4' in col]
product_col_3 = [col for col in df_train.columns if '_ult1_3' in col]
product_col_2 = [col for col in df_train.columns if '_ult1_2' in col]
product_col_1 = [col for col in df_train.columns if '_ult1_1' in col]

df_train['tot5'] = df_train[product_col_5].sum(axis=1)
df_test['tot5'] = df_test[product_col_5].sum(axis=1)

In [15]:
for col in product_col[2:]:
    df_train[col+'_past'] = (df_train[col+'_5']+df_train[col+'_4']+df_train[col+'_3']+df_train[col+'_2']+df_train[col+'_1'])/5
    df_test[col+'_past'] = (df_test[col+'_5']+df_test[col+'_4']+df_test[col+'_3']+df_test[col+'_2']+df_test[col+'_1'])/5

In [16]:
for pro in product_col[2:]:
    df_train[pro+'_past'] = df_train[pro+'_past']*(1-df_train[pro+'_5'])
    df_test[pro+'_past'] = df_test[pro+'_past']*(1-df_test[pro+'_5'])

In [ ]:
model1 = Sequential()
model1.add(Bidirectional(LSTM(150, input_shape=(5, 22), dropout_W=0.0, dropout_U=0.3)))

model2 = Sequential()
model2.add(Dense(150, input_dim=len(col2), init='uniform', activation='relu'))
model2.add(Dropout(0.3))

merged = Merge([model1,model2], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(64, init='uniform', activation='relu'))
final_model.add(Dropout(0.3))
final_model.add(Dense(22, init='uniform', activation='softmax'))
final_model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['categorical_accuracy'])

final_model.fit([x_train1, x_train2], y_train, validation_split=0.2, nb_epoch=170, batch_size=100, verbose = 2)

Train on 45679 samples, validate on 1228 samples
Epoch 1/100
0s - loss: 1.2040 - categorical_accuracy: 0.8137
Epoch 2/100
0s - loss: 1.0435 - categorical_accuracy: 0.8450
Epoch 3/100
0s - loss: 1.0182 - categorical_accuracy: 0.8461
Epoch 4/100
0s - loss: 1.0051 - categorical_accuracy: 0.8492
Epoch 5/100
0s - loss: 0.9964 - categorical_accuracy: 0.8500
Epoch 6/100
0s - loss: 0.9906 - categorical_accuracy: 0.8508
Epoch 7/100
0s - loss: 0.9856 - categorical_accuracy: 0.8501
Epoch 8/100
0s - loss: 0.9820 - categorical_accuracy: 0.8511
Epoch 9/100
0s - loss: 0.9786 - categorical_accuracy: 0.8534
Epoch 10/100
0s - loss: 0.9758 - categorical_accuracy: 0.8531
Epoch 11/100
0s - loss: 0.9733 - categorical_accuracy: 0.8535
Epoch 12/100
0s - loss: 0.9710 - categorical_accuracy: 0.8542
Epoch 13/100
0s - loss: 0.9688 - categorical_accuracy: 0.8537
Epoch 14/100
0s - loss: 0.9672 - categorical_accuracy: 0.8543
Epoch 15/100
0s - loss: 0.9654 - categorical_accuracy: 0.8558
Epoch 16/100
0s - loss: 0.9641

In [ ]:
final_model.save("history_model.h5")

In [ ]:
y_pred = final_model.predict(x_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

Build LSTM model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               24047616  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 11485)             5891805   
_________________________________________________________________
activation_1 (Activation)    (None, 11485)             0         
Total params: 29,939,421
Trainable params: 29,939,421
Non-trainable params: 0
_________________________________________________________________


In [3]:
num_max = 1000
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')

In [ ]:
num_max = 1000
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
le = LabelEncoder()
tags = le.fit_transform(tags)

In [35]:
# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 2

In [80]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(256, dropout=0.2)))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')

Train...


In [84]:
model.fit(texts_mat,tags,batch_size=32,epochs=3,verbose=1,validation_split=0.2)

Train on 2880000 samples, validate on 720000 samples
Epoch 1/3
2880000/2880000 [==============================] - 62896s 22ms/step - loss: 0.0579 - acc: 0.9748 - val_loss: 0.0461 - val_acc: 0.9808
Epoch 2/3
2880000/2880000 [==============================] - 62227s 22ms/step - loss: 0.0470 - acc: 0.9802 - val_loss: 0.0425 - val_acc: 0.9825
Epoch 3/3
2880000/2880000 [==============================] - 61540s 21ms/step - loss: 0.0464 - acc: 0.9806 - val_loss: 0.0425 - val_acc: 0.9823


In [91]:
scores = model.evaluate(test_cnn_texts_mat,test_tags)

400000/400000 [==============================] - 2655s 7ms/step


In [92]:
print("Model accuracy on test data is",scores[1]*100,"percent")

Model accuracy on test data is 94.84825000000001 percent


In [96]:
#import cythonmagic
model.save_weights("D:\\Azim\\LSTM_Bidirectional\\model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
merged = Merge([final_model,model], mode='concat')

scores = model.evaluate(test_cnn_texts_mat,test_tags)
print("Combiend accuracy is ", scores[1]*100,"percent")

Combiend accuracy is 92.768285000000001 percent
